In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
movies = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")

In [3]:
print(movies.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


In [4]:
print(ratings.head())

   userId  movieId  rating   timestamp
0       1      296     5.0  1147880044
1       1      306     3.5  1147868817
2       1      307     5.0  1147868828
3       1      665     5.0  1147878820
4       1      899     3.5  1147868510


In [5]:
print(len(ratings))

25000095


In [6]:
user_counts = ratings['userId'].value_counts()
movie_counts = ratings['movieId'].value_counts()
ratings = ratings[
    ratings['userId'].isin(user_counts[user_counts >= 3].index) &
    ratings['movieId'].isin(movie_counts[movie_counts >= 3].index)
]

In [7]:
print(len(ratings))

24974531


In [55]:
print("Có giá trị null trong ratings:", ratings.isnull().sum().sum())

Có giá trị null trong ratings: 0


In [8]:
np.random.seed(3)
msk = np.random.rand(len(ratings)) < 0.8
train = ratings[msk].copy()
val = ratings[~msk].copy()
print(len(train), len(val))

19979459 4995072


In [9]:
# Bước 4: Tạo ma trận thưa người dùng-phim
user_ids = train['userId'].unique()
movie_ids = train['movieId'].unique()
user_id_map = {uid: i for i, uid in enumerate(user_ids)}
movie_id_map = {mid: i for i, mid in enumerate(movie_ids)}

In [10]:
rows = [user_id_map[uid] for uid in train['userId']]
cols = [movie_id_map[mid] for mid in train['movieId']]
data = train['rating'].values

In [11]:
global_mean = train['rating'].mean()
user_means = train.groupby('userId')['rating'].mean().to_dict()
item_means = train.groupby('movieId')['rating'].mean().to_dict()
data_normalized = [
    r - (user_means.get(uid, global_mean) + item_means.get(mid, global_mean) - global_mean)
    for uid, mid, r in zip(train['userId'], train['movieId'], train['rating'])
]
train_matrix = csr_matrix((data_normalized, (rows, cols)), shape=(len(user_ids), len(movie_ids)))
print(train_matrix.shape)
print("Số phần tử khác 0:", train_matrix.nnz)
print("Tỷ lệ thưa:", 1 - train_matrix.nnz / (train_matrix.shape[0] * train_matrix.shape[1]))

(162541, 41086)
Số phần tử khác 0: 19979459
Tỷ lệ thưa: 0.9970082387337088


In [12]:
n_components = 200
svd = TruncatedSVD(n_components=n_components, random_state=42)
print("\nBắt đầu huấn luyện TruncatedSVD...")
train_matrix_svd = svd.fit_transform(train_matrix)
print("Huấn luyện hoàn tất.")
print(f"Kích thước ma trận sau SVD: {train_matrix_svd.shape}")
print("Tỷ lệ phương sai:", sum(svd.explained_variance_ratio_))


Bắt đầu huấn luyện TruncatedSVD...
Huấn luyện hoàn tất.
Kích thước ma trận sau SVD: (162541, 200)
Tỷ lệ phương sai: 0.2968308959464928


In [13]:
val_true = []
val_pred = []
for _, row in val.iterrows():
    user_id = row['userId']
    movie_id = row['movieId']
    rating = row['rating']
    if user_id in user_id_map and movie_id in movie_id_map:
        user_idx = user_id_map[user_id]
        movie_idx = movie_id_map[movie_id]
        pred = np.dot(train_matrix_svd[user_idx], svd.components_[:, movie_idx])
        pred += user_means.get(user_id, global_mean) + item_means.get(movie_id, global_mean) - global_mean
        pred = np.clip(pred, 0.5, 5.0)
        val_true.append(rating)
        val_pred.append(pred)
rmse = np.sqrt(mean_squared_error(val_true, val_pred)) if val_true else float('inf')

In [14]:
print(f"\nRMSE-validation: {rmse:.3f}")
print("Số lượng đánh giá được so sánh:", len(val_true))
for true, pred in zip(val_true[:10], val_pred[:10]):
    print(f"Thực tế: {true:.3f}, Dự đoán: {pred:.3f}")
print(f"Phương sai của dự đoán: {np.var(val_pred):.3f}")
print(f"Phương sai của thực tế: {np.var(val_true):.3f}")


RMSE-validation: 0.847
Số lượng đánh giá được so sánh: 4994974
Thực tế: 3.500, Dự đoán: 4.334
Thực tế: 4.000, Dự đoán: 3.537
Thực tế: 4.500, Dự đoán: 4.423
Thực tế: 4.000, Dự đoán: 4.393
Thực tế: 2.000, Dự đoán: 3.570
Thực tế: 5.000, Dự đoán: 4.092
Thực tế: 2.000, Dự đoán: 3.450
Thực tế: 3.500, Dự đoán: 3.556
Thực tế: 4.500, Dự đoán: 3.991
Thực tế: 3.500, Dự đoán: 3.606
Phương sai của dự đoán: 0.501
Phương sai của thực tế: 1.123
